Стартовая настроечка: про кого смотрим?
* A - Артём
* M - Маша

In [ ]:
Person = "A"
Person = "M"

Ещё одна настроечка: где ищем эксельки (для тех вузов, по которым нифига онлайн не парсится) и куда складываем результат?

In [ ]:
path = "C:\\Users\\Ally\\Desktop\\Маша\\"

Дальше уже не настройки, а сама прога

In [ ]:
Docs = {"A": "111-111-111 11", "M": "222-222-222 22"}  # номера СНИЛС
Marks = {"M": 233, "A": 250}
Paths = {
    "M": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/12.03.02.pdf",
    "A": "https://priem.bmstu.ru/lists/upload/enrollees/first/moscow-1/09.03.01.pdf"
}
Places = {"M": 17, "A": 101}

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# pip install tabula-py

In [ ]:
from tabula import read_pdf
bmstu_path = Paths[Person]
df = read_pdf(bmstu_path, pages='all')
len(df)

Парсим все данные по конкурсу на нашу специальность

In [ ]:
frames = []
for i in range(len(df)):
    first_valid = 0
    df_tmp = df[i]
    try:
        scip_flag = (df[i].loc[0][0] == "No" or np.isnan(df[i].loc[0][0]))
    except Exception as e:
        scip_flag = True
#     print("first_item=", df[i].loc[0][0], "len_df=",len(df[i]))
    if scip_flag:
        for j in range(len(df[i])):
            try:
                if df[i].loc[j][0].isnumeric():
                    if first_valid == 0:
                        first_valid = j
            except Exception as e:
               str1 = "we have found an exception"
#     print("first_valid=",first_valid)
    df_tmp.columns = df[2].columns
    df_tmp = df_tmp[first_valid:]
    df_tmp = df_tmp.reset_index(drop=True)
    frames.append(df_tmp)
df_concurs = pd.concat(frames, sort=False)
df_concurs = df_concurs.reset_index(drop=True)
df_concurs

In [ ]:
num = df_concurs[df_concurs["СНИЛС /\rРег. номер"] == Docs[Person]].index.values[0] + 1
print(
    "Искомый абитуриент сейчас \n\033[1m\033[34m"
    + str(num)
    + "\033[0m из "
    + str(len(df_concurs))
    + ", участвующих в этом конкурсе"
)

In [ ]:
df_concurs_originals = df_concurs[df_concurs["Ориг."] == "Да"]
df_concurs_originals = df_concurs_originals.reset_index(drop=True)
# df_concurs_originals

In [ ]:
num = df_concurs_originals[df_concurs_originals["СНИЛС /\rРег. номер"] == Docs[Person]].index.values[0] + 1
print(
    "Искомый абитуриент на текущий момент \n\033[1m\033[34m"
    + str(num)
    + "\033[0m из "
    + str(len(df_concurs_originals))
    + ", принесших в вуз Оригинал"
)

In [ ]:
df_concurs_agree = df_concurs_originals[df_concurs_originals["Согл."] == "Да"]
df_concurs_agree = df_concurs_agree.reset_index(drop=True)
# df_concurs_agree

In [ ]:
num_prev = df_concurs_agree[df_concurs_agree["Сумма\rбаллов"].astype(int) == 
                 min(
                     df_concurs_agree[
                         df_concurs_agree["Сумма\rбаллов"].astype(int) > Marks[Person]
                     ]["Сумма\rбаллов"].astype(int)
                 )].index.values[0] + 1

In [ ]:
try:
    num = df_concurs_agree[df_concurs_agree["СНИЛС /\rРег. номер"] == Docs[Person]].index.values[0] + 1
    print(
        "Искомый абитуриент на текущий момент \n\033[1m\033[34m"
        + str(num)
        + "\033[0m из "
    + str(Places[Person]) + " имеющихся мест для зачисления \n\n"
    + "Всего на это направление сейчас принесли в вуз и Оригинал, и Согласие на зачисление "
        + str(len(df_concurs_agree)) + " абитуриентов"
    )
except:
    print(
        "Искомый абитуриент на текущий момент не подал Согласие на зачисление\n"
        + "Подаст - будет \033[1m\033[34m"
    + str(num_prev+1)
    + "\033[0m из "
    + str(Places[Person]) + " имеющихся мест для зачисления \n\n"
    + "Всего на это направление сейчас принесли в вуз и Оригинал, и Согласие на зачисление "
        + str(len(df_concurs_agree)) + " абитуриентов"
    )

### Поиск подлинников

(где получится - по онлайн-данным, где не получится - по сохраненным в excel)

Нишмагли проанализировать вообще:
* МГУ (публикуется только уникальный номер конкурсного заявления, ни СНИЛСа, ни ФИО в списках нет)
* ИТМО (публикуется только номер конкурсного заявления, ни СНИЛСа, ни ФИО в списках нет)

**ВШЭ**

In [ ]:
hse_dict = {
    "01.03.02": "https://enrol.hse.ru/storage/public_report_2022/moscow/Bachelors/KS_M_B_OM_M_PMiI.xlsx",
    "01.03.04": "https://enrol.hse.ru/storage/public_report_2022/moscow/Bachelors/KS_M_B_OM_M_PM.xlsx",
    "09.03.01": "https://enrol.hse.ru/storage/public_report_2022/moscow/Bachelors/KS_M_B_OM_M_IVT.xlsx",
}

In [ ]:
hse_spec = {}
for spec in hse_dict:
    try:
        hse_spec[spec] = pd.read_excel(hse_dict[spec])
        hse_spec[spec].columns = hse_spec[spec].iloc[14]
        hse_spec[spec] = hse_spec[spec][36:]
        hse_spec[spec] = hse_spec[spec].reset_index(drop=True)
        hse_spec[spec] = hse_spec[spec].iloc[:,[i%2==0 for i in range(len(hse_spec[spec].columns))]]
        hse_spec[spec]["ВШЭ"] = spec
    except Exception as e:
       print("пропустили", hse_dict[spec], "-", e)
# len(hse_spec)

In [ ]:
hse_originals = []
i = 0
for spec in hse_spec:
    hse_originals.append(hse_spec[spec][hse_spec[spec]["Оригинал документа об образовании"] == "Да"])
    hse_originals[i] = hse_originals[i].reset_index(drop=True)
# len(hse_originals)

In [ ]:
hse = pd.DataFrame({})
for i in hse_originals:
    hse = pd.concat([hse,i], ignore_index= True, sort=False)
hse = hse.drop_duplicates()
hse = hse.reset_index(drop=True)
len(hse)

**МИФИ**

In [ ]:
mephi_dict = {
    "01.03.02иикс": "https://org.mephi.ru/pupil-rating/get-rating/entity/9045/original/no",
    "01.03.02иикс_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9738/original/no",
    "01.03.02лаплаз": "https://org.mephi.ru/pupil-rating/get-rating/entity/9043/original/no",
    "01.03.02лаплаз_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9737/original/no",
    "03.03.01": "https://org.mephi.ru/pupil-rating/get-rating/entity/9047/original/no",
    "03.03.01_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9739/original/no",
    "03.03.02": "https://org.mephi.ru/pupil-rating/get-rating/entity/9050/original/no",
    "03.03.02_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9740/original/no",
    "09.03.00": "https://org.mephi.ru/pupil-rating/get-rating/entity/9052/original/no",
    "09.03.01_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9741/original/no",
    "10.03.01иикс": "https://org.mephi.ru/pupil-rating/get-rating/entity/9054/original/no",
    "10.03.01иикс_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9743/original/no",
    "10.03.01ифтэб": "https://org.mephi.ru/pupil-rating/get-rating/entity/9729/original/no",
    "10.03.01ифтэб_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9744/original/no",
    "10.05.00": "https://org.mephi.ru/pupil-rating/get-rating/entity/9087/original/no",
    "10.05.04_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9761/original/no",
    "10.05.05_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9762/original/no",
    "11.03.04": "https://org.mephi.ru/pupil-rating/get-rating/entity/9056/original/no",
    "11.03.04_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9745/original/no",
    "12.03.01": "https://org.mephi.ru/pupil-rating/get-rating/entity/9059/original/no",
    "12.03.01_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9746/original/no",
    "12.03.03": "https://org.mephi.ru/pupil-rating/get-rating/entity/9060/original/no",
    "12.03.03_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9747/original/no",
    "12.03.04": "https://org.mephi.ru/pupil-rating/get-rating/entity/9062/original/no",
    "12.03.04_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9748/original/no",
    "12.03.05": "https://org.mephi.ru/pupil-rating/get-rating/entity/9064/original/no",
    "12.03.05_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9749/original/no",
    "15.03.00": "https://org.mephi.ru/pupil-rating/get-rating/entity/9068/original/no",
    "15.03.06_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9753/original/no",
    "22.03.01": "https://org.mephi.ru/pupil-rating/get-rating/entity/9072/original/no",
    "22.03.01_пл": "https://org.mephi.ru/pupil-rating/get-rating/entity/9755/original/no",
}

In [ ]:
mephi_spec = {}
for spec in mephi_dict:
    try:
        read_html = pd.read_html(mephi_dict[spec], encoding="utf-8")
        mephi_spec[spec] = read_html[0]
        for i in range(mephi_spec[spec].columns.nlevels-1, -1, -1):
            if i != 1:
                mephi_spec[spec] = mephi_spec[spec].droplevel(i, axis=1)
        mephi_spec[spec]["МИФИ"] = spec
    except Exception as e:
       print("пропустили", mephi_dict[spec], "-", e)
# len(mephi_spec)

In [ ]:
mephi_originals = []
i = 0
for spec in mephi_spec:
    mephi_originals.append(mephi_spec[spec][mephi_spec[spec]["Документы"] == "Оригинал"])
    mephi_originals[i] = mephi_originals[i].reset_index(drop=True)
# len(mephi_originals)

In [ ]:
mephi = pd.DataFrame({})
for i in mephi_originals:
    mephi = pd.concat([mephi,i], ignore_index= True, sort=False)
mephi = mephi.drop_duplicates()
mephi = mephi.reset_index(drop=True)
len(mephi)

**МФТИ**

In [ ]:
mipt_dict = {
    "01.03.02": path + "МФТИ 01.03.02.xlsx",
    "03.03.01": path + "МФТИ 03.03.01.xlsx",
    "09.03.01": path + "МФТИ 09.03.01.xlsx",
    "10.05.01": path + "МФТИ 01.03.02.xlsx",
    "11.03.04": path + "МФТИ 03.03.01.xlsx",
    "16.03.01": path + "МФТИ 09.03.01.xlsx",
    "19.03.01": path + "МФТИ 09.03.01.xlsx",
    "27.03.03": path + "МФТИ 09.03.01.xlsx"    
}

In [ ]:
mipt_spec = {}
for spec in mipt_dict:
    try:
        mipt_spec[spec] = pd.read_excel(mipt_dict[spec])
        mipt_spec[spec]["МФТИ"] = spec
    except Exception as e:
       print("пропустили", mipt_dict[spec], "-", e)
# len(hse_spec)

In [ ]:
mipt_originals = []
i = 0
for spec in mipt_spec:
    mipt_originals.append(mipt_spec[spec][mipt_spec[spec]["ДО"] == "Оригинал"])
    mipt_originals[i] = mipt_originals[i].reset_index(drop=True)
# len(mipt_originals)

In [ ]:
mipt = pd.DataFrame({})
for i in mipt_originals:
    mipt = pd.concat([mipt,i], ignore_index= True, sort=False)
mipt = mipt.drop_duplicates()
mipt = mipt.reset_index(drop=True)
len(mipt)

**МАИ**

In [ ]:
mai_dict = {
    "01.03.00": path + "МАИ 01.03.00.xlsx",
    "02.03.02": path + "МАИ 02.03.02.xlsx",
    "09.03.01": path + "МАИ 09.03.01.xlsx",
    "09.03.02": path + "МАИ 09.03.02.xlsx",
    "09.03.03": path + "МАИ 09.03.03.xlsx",
    "10.03.01": path + "МАИ 10.03.01.xlsx",
    "11.03.01": path + "МАИ 11.03.01.xlsx",
    "11.03.02": path + "МАИ 11.03.02.xlsx",
    "11.03.03": path + "МАИ 11.03.03.xlsx",
    "12.03.04": path + "МАИ 12.03.04.xlsx",
    "13.03.02": path + "МАИ 13.03.02.xlsx",
    "15.03.03": path + "МАИ 15.03.03.xlsx",
    "22.03.00": path + "МАИ 22.03.00.xlsx",
}

In [ ]:
mai_spec = {}
for spec in mai_dict:
    try:
        mai_spec[spec] = pd.read_excel(mai_dict[spec])
        mai_spec[spec]["МАИ"] = spec
    except Exception as e:
       print("пропустили", mai_dict[spec], "-", e)
# len(mai_spec)

In [ ]:
mai_originals = []
i = 0
for spec in mai_spec:
    mai_originals.append(mai_spec[spec][mai_spec[spec]["Подлинник или копия документа об образовании"] == "Подлинник"])
    mai_originals[i] = mai_originals[i].reset_index(drop=True)
# len(mai_originals)

In [ ]:
mai = pd.DataFrame({})
for i in mai_originals:
    mai = pd.concat([mai,i], ignore_index= True, sort=False)
mai = mai.drop_duplicates()
mai = mai.reset_index(drop=True)
# len(mai)
mai

**МЭИ**

In [ ]:
mpei_dict = {
    "01.03.02": "https://pk.mpei.ru/inform/list581bacc.html",
    "01.03.02_пл7": "https://pk.mpei.ru/inform/list582bacpa.html",
    "01.03.02_пл14": "https://pk.mpei.ru/inform/list582bacpb.html",
    "01.03.02_пл28": "https://pk.mpei.ru/inform/list582bacpc.html",
    "01.03.02мм": "https://pk.mpei.ru/inform/list16bacc.html",
    "01.03.02мм_пл7": "https://pk.mpei.ru/inform/list48bacpa.html",
    "01.03.02мм_пл14": "https://pk.mpei.ru/inform/list48bacpb.html",
    "01.03.02мм_пл28": "https://pk.mpei.ru/inform/list48bacpc.html",
    "09.03.01": "https://pk.mpei.ru/inform/list16bacc.html",
    "09.03.01_пл7": "https://pk.mpei.ru/inform/list49bacpa.html",
    "09.03.01_пл14": "https://pk.mpei.ru/inform/list49bacpb.html",
    "09.03.01_пл28": "https://pk.mpei.ru/inform/list49bacpc.html",
    "09.03.03": "https://pk.mpei.ru/inform/list35bacc.html",
    "09.03.03_пл7": "https://pk.mpei.ru/inform/list56bacpa.html",
    "09.03.03_пл14": "https://pk.mpei.ru/inform/list56bacpb.html",
    "09.03.03_пл28": "https://pk.mpei.ru/inform/list56bacpc.html",
    "10.03.01": "https://pk.mpei.ru/inform/list22bacc.html",
    "10.03.01_пл7": "https://pk.mpei.ru/inform/list57bacpa.html",
    "10.03.01_пл14": "https://pk.mpei.ru/inform/list57bacpb.html",
    "10.03.01_пл28": "https://pk.mpei.ru/inform/list57bacpc.html",
    "11.03.01": "https://pk.mpei.ru/inform/list19bacc.html",
    "11.03.01_пл7": "https://pk.mpei.ru/inform/list53bacpa.html",
    "11.03.01_пл14": "https://pk.mpei.ru/inform/list53bacpb.html",
    "11.03.01_пл28": "https://pk.mpei.ru/inform/list53bacpc.html",
    "11.03.04": "https://pk.mpei.ru/inform/list10bacc.html",
    "11.03.04_пл7": "https://pk.mpei.ru/inform/list47bacpa.html",
    "11.03.04_пл14": "https://pk.mpei.ru/inform/list47bacpb.html",
    "11.03.04_пл28": "https://pk.mpei.ru/inform/list47bacpc.html",
    "11.03.04нн": "https://pk.mpei.ru/inform/list959bacc.html",
    "11.03.04нн_пл7": "https://pk.mpei.ru/inform/list960bacpa.html",
    "11.03.04нн_пл14": "https://pk.mpei.ru/inform/list960bacpb.html",
    "11.03.04нн_пл28": "https://pk.mpei.ru/inform/list960bacpc.html",
    "11.05.01": "https://pk.mpei.ru/inform/list20bacc.html",
    "11.05.01_пл7": "https://pk.mpei.ru/inform/list55bacpa.html",
    "11.05.01_пл14": "https://pk.mpei.ru/inform/list55bacpb.html",
    "11.05.01_пл28": "https://pk.mpei.ru/inform/list55bacpc.html",
    "12.03.04": "https://pk.mpei.ru/inform/list15bacc.html",
    "12.03.04_пл7": "https://pk.mpei.ru/inform/list51bacpa.html",
    "12.03.04_пл14": "https://pk.mpei.ru/inform/list51bacpb.html",
    "12.03.04_пл28": "https://pk.mpei.ru/inform/list51bacpc.html",
    "15.03.01": "https://pk.mpei.ru/inform/list1bacc.html",
    "15.03.01_пл7": "https://pk.mpei.ru/inform/list33bacpa.html",
    "15.03.01_пл14": "https://pk.mpei.ru/inform/list33bacpb.html",
    "15.03.01_пл28": "https://pk.mpei.ru/inform/list33bacpc.html"
}

In [ ]:
mpei_spec = {}
for spec in mpei_dict:
#     print(mpei_dict[spec])
    try:
        read_html = pd.read_html(mpei_dict[spec])
        if len(read_html) == 2:
            mpei_spec[spec] = pd.concat([read_html[0], read_html[1]], sort=False)
        else:
            mpei_spec[spec] = read_html[0]
        mpei_spec[spec].columns = mpei_spec[spec].iloc[1]
        mpei_spec[spec] = mpei_spec[spec][2:]
        mpei_spec[spec] = mpei_spec[spec].reset_index(drop=True)
        mpei_spec[spec]["СНИЛС или Рег.номер"] = mpei_spec[spec]["СНИЛС или Рег.номер"].replace(
            to_replace=r"(\w*:\s*)(\d{3})(\d{3})(\d{3})(\d{2})", 
            value=r"\2-\3-\4 \5", 
            regex=True)
        mpei_spec[spec]["МЭИ"] = spec
    except Exception as e:
       print("пропустили", mpei_dict[spec], "-", e)
# len(mpei_spec)

In [ ]:
mpei_agree = []
i = 0
for spec in mpei_spec:
    mpei_agree.append(mpei_spec[spec][mpei_spec[spec]["Согласие"] == "подано"])
    mpei_agree[i] = mpei_agree[i].reset_index(drop=True)
# len(mpei_agree)

In [ ]:
mpei = pd.DataFrame({})
for i in mpei_agree:
    mpei = pd.concat([mpei,i], ignore_index= True, sort=False)
mpei = mpei.drop_duplicates()
mpei = mpei.reset_index(drop=True)
len(mpei)

**МИСиС**

In [ ]:
misis_dict = {
    "01.03.04": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-010304",
    "01.03.04_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-010304",
    "03.03.02": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-030302",
    "03.03.02_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-030302",
    "09.03.00": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-090300",
    "09.03.00_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-090300",
    "11.03.04": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-110304",
    "11.03.04_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-110304",
    "13.03.02": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-150302",
    "13.03.02_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-130302",
    "15.03.02": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-150302",
    "15.03.02_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-150302",
    "21.05.00": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=SPEC-BUDJ-O-210500",
    "21.05.00_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=SPEC-COMM-O-210500",
    "22.03.01": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-220301",
    "22.03.01_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-220301",
    "28.03.00": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-BUDJ-O-280300",
    "28.03.00_пл": "https://misis.ru/applicants/admission/progress/baccalaureate-and-specialties/list-of-applicants/list/?id=BAC-COMM-O-280300"
}

In [ ]:
misis_spec = {}
for spec in misis_dict:
    try:
        read_html = pd.read_html(misis_dict[spec])
        misis_spec[spec] = read_html[0]
        misis_spec[spec]["МИСиС"] = spec
        misis_spec[spec] = misis_spec[spec].droplevel(1, axis=1)
    except Exception as e:
       print("пропустили", misis_dict[spec], "-", e)
# len(misis_spec)

In [ ]:
misis_originals = []
i = 0
for spec in misis_spec:
    misis_originals.append(misis_spec[spec][misis_spec[spec]["Оригинал"] == "+"])
    misis_originals[i] = misis_originals[i].reset_index(drop=True)
# len(misis_originals)

In [ ]:
misis = pd.DataFrame({})
for i in misis_originals:
    misis = pd.concat([misis,i], ignore_index= True, sort=False)
misis = misis.drop_duplicates()
misis = misis.reset_index(drop=True)
len(misis)

**МИРЭА**

In [ ]:
mirea_dict = {
    "01.03.02": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712416774837808438",
    "01.03.02_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712416774839905590",
    "01.03.04": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712416794849881398",
    "01.03.04_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712416794851978550",
    "09.03.01иии": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417222681472310",
    "09.03.01иии_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417222683569462",
    "09.03.01иит": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712416831985200438",
    "09.03.01иит_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712416831987297590",
    "09.03.02икб": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417235799158070",
    "09.03.02икб_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417235801255222",
    "09.03.02ири": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417616938708278",
    "09.03.02ири_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417616940805430",
    "09.03.02кд": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417608956947766",
    "09.03.02кд_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417608959044918",
    "09.03.02фр": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1714956156281072950",
    "09.03.02фр_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1714956156283170102",
    "09.03.03": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417600979381558",
    "09.03.03_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417600981478710",
    "09.03.04": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417591424757046",
    "09.03.04_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417591426854198",
    "10.03.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417582796025142",
    "10.03.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417582798122294",
    "11.03.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417535526219062",
    "11.03.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417535528316214",
    "11.03.02": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417522705280310",
    "11.03.02_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417522707377462",
    "11.03.03": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417515145047350",
    "11.03.03_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417515147144502",
    "11.03.04": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417506900094262",
    "11.03.04_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417506902191414",
    "11.05.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417497502756150",
    "11.05.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417497504853302",
    "12.03.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417490279116086",
    "12.03.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417490281213238",
    "12.03.04": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417482846809398",
    "12.03.04_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417482848906550",
    "12.03.05": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417475758435638",
    "12.03.05_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417475760532790",
    "12.05.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417468490755382",
    "12.05.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417468492852534",
    "15.03.06": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417442958978358",
    "15.03.06_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417442961075510",
    "22.03.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417406008208694",
    "22.03.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417406010305846",
    "27.03.05": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417380017155382",
    "27.03.05_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417380019252534",
    "28.03.01": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417372125572406",
    "28.03.01_пл": "https://priem.mirea.ru/accepted-entrants-list/personal_code_rating.php?competition=1712417372127669558"
}

In [ ]:
mirea_spec = {}
for spec in mirea_dict:
    try:
        read_html = pd.read_html(mirea_dict[spec])
        mirea_spec[spec] = read_html[0]
        mirea_spec[spec]["МИРЭА"] = spec
        mirea_spec[spec]["СНИЛС/уникальный номер"] = mirea_spec[spec]["СНИЛС/уникальный номер"].replace(
            to_replace=r"(\d{3}-\d{3}-\d{3})-(\d{2})", 
            value=r"\1 \2", 
            regex=True)
    except Exception as e:
       print("пропустили", mirea_dict[spec], "-", e)
# len(mirea_spec)

In [ ]:
mirea_originals = []
i = 0
for spec in mirea_spec:
    mirea_originals.append(mirea_spec[spec][mirea_spec[spec]["Наличие подлинника/отметки о подлиннике с ЕПГУ"] == "да"])
    mirea_originals[i] = mirea_originals[i].reset_index(drop=True)
# len(mirea_originals)

In [ ]:
mirea = pd.DataFrame({})
for i in mirea_originals:
    mirea = pd.concat([mirea,i], ignore_index= True, sort=False)
mirea = mirea.drop_duplicates()
mirea = mirea.reset_index(drop=True)
len(mirea)

**МИИГАиК**

In [ ]:
miigaik_dict = {
    "09.03.02": path + "МИИГАиК 09.03.02.xlsx",
    "09.03.03": path + "МИИГАиК 09.03.03.xlsx",
    "12.03.02": path + "МИИГАиК 12.03.02.xlsx",
    "12.03.05": path + "МИИГАиК 12.03.05.xlsx"
}

In [ ]:
miigaik_spec = {}
for spec in miigaik_dict:
    try:
        miigaik_spec[spec] = pd.read_excel(miigaik_dict[spec])
        miigaik_spec[spec]["МИИГАиК"] = spec
    except Exception as e:
       print("пропустили", miigaik_dict[spec], "-", e)
# len(miigaik_spec)

In [ ]:
miigaik_originals = []
i = 0
for spec in miigaik_spec:
    miigaik_originals.append(miigaik_spec[spec][miigaik_spec[spec]["Тип документа"] == "Оригинал"])
    miigaik_originals[i] = miigaik_originals[i].reset_index(drop=True)
# len(miigaik_originals)

In [ ]:
miigaik = pd.DataFrame({})
for i in miigaik_originals:
    miigaik = pd.concat([miigaik,i], ignore_index= True, sort=False)
miigaik = miigaik.drop_duplicates()
miigaik = miigaik.reset_index(drop=True)
len(miigaik)

**РЭУ**

In [ ]:
rea_dict = {
    "01.03.02": "https://abitlist.rea.ru/Home/ViewList?listId=B12DA8D4",
    "01.03.02_пл": "https://abitlist.rea.ru/Home/ViewList?listId=BE3438AC",
    "09.03.02": "https://abitlist.rea.ru/Home/ViewList?listId=85D92D63",
    "09.03.02_пл": "https://abitlist.rea.ru/Home/ViewList?listId=EA9BE83F",
    "09.03.03": "https://abitlist.rea.ru/Home/ViewList?listId=784461D9",
    "09.03.03_пл": "https://abitlist.rea.ru/Home/ViewList?listId=3EFFB49E",
    "10.03.01": "https://abitlist.rea.ru/Home/ViewList?listId=1F6F5D1E",
    "10.03.01_пл": "https://abitlist.rea.ru/Home/ViewList?listId=6C683A8F",
    "15.03.02": "https://abitlist.rea.ru/Home/ViewList?listId=BC5F4B67",
    "15.03.02_пл": "https://abitlist.rea.ru/Home/ViewList?listId=84F01F3C",
    "27.03.05_пл": "https://abitlist.rea.ru/Home/ViewList?listId=592B36B4"
}

In [ ]:
rea_spec = {}
for spec in rea_dict:
    try:
        read_html = pd.read_html(rea_dict[spec])
        rea_spec[spec] = read_html[0]
        rea_spec[spec]["РЭУ"] = spec
        rea_spec[spec]["СНИЛС"] = rea_spec[spec]["СНИЛС"].replace(
            to_replace=r"(\d{3})\D*(\d{3})\D*(\d{3})\D*(\d{2})", 
            value=r"\1-\2-\3 \4", 
            regex=True)
    except Exception as e:
       print("пропустили", rea_dict[spec], "-", e)
# len(rea_spec)

In [ ]:
rea_originals = []
i = 0
for spec in rea_spec:
    rea_originals.append(rea_spec[spec][rea_spec[spec]["Оригинал"] == "Да"])
    rea_originals[i] = rea_originals[i].reset_index(drop=True)
# len(rea_originals)

In [ ]:
rea = pd.DataFrame({})
for i in rea_originals:
    rea = pd.concat([rea,i], ignore_index= True, sort=False)
rea = rea.drop_duplicates()
rea = rea.reset_index(drop=True)
len(rea)

**1-й Мед**

(пока выключила, всё равно никого не находится)

In [ ]:
# онлайн forbidden, сохранять отсюда: 
# https://www.sechenov.ru/admissions/priemnaya-kampaniya-2022/ranzhirovannye-spiski-postupayushchikh-2022-2023/
sechenov_path = path + "sechenov.pdf"
sechenov_df = read_pdf(sechenov_path, pages='all')
# len(sechenov_df)

In [ ]:
pages = []
for i in range(len(sechenov_df)):
    pages.append(sechenov_df[i])
sechenov_concurs = pd.concat(pages, sort=False)
sechenov_concurs.columns = sechenov_concurs.iloc[0]
sechenov_concurs = sechenov_concurs[2:]
sechenov_concurs["Идентифика\rтор"] = sechenov_concurs["Идентифика\rтор"].replace(
    to_replace=r"(\d{3}-\d{3}-\d{3})(\s*)(\d{2})", 
    value=r"\1 \3", 
    regex=True)
sechenov_concurs["1Мед"] = "Да"
sechenov_originals = sechenov_concurs[sechenov_concurs["Наличие\rоригинала\rда / нет"]=="Да"]
# sechenov_originals["1Мед"] = "Да"
sechenov_originals = sechenov_originals.reset_index(drop=True)
len(sechenov_originals)

**СПбГУ**

In [ ]:
spbu_dict = {
    "01.03.01_5000": "https://cabinet.spbu.ru/Lists/1k_Rating/list_8ea405e1-7f6e-4ff7-b0bd-fc9d2647bb54.html",
    "01.03.02_5005": "https://cabinet.spbu.ru/Lists/1k_Rating/list_2df3cab7-781a-4467-be96-3302b6f105ae.html",
    "01.03.02_5156": "https://cabinet.spbu.ru/Lists/1k_Rating/list_071a53e5-c6b0-4134-a89f-793db1c177d9.html",
    "01.03.02_5231": "https://cabinet.spbu.ru/Lists/1k_Rating/list_2257c623-1e5e-4c86-b0af-655de0a99871.html",
    "01.03.03_5008": "https://cabinet.spbu.ru/Lists/1k_Rating/list_846c2a25-562d-4005-9a39-a54804b939e7.html",
    "01.05.01_5088": "https://cabinet.spbu.ru/Lists/1k_Rating/list_0aa95bd2-8183-4273-a61f-88fdab9f9d16.html",
    "01.05.01_5089": "https://cabinet.spbu.ru/Lists/1k_Rating/list_b9167a2a-6eae-4b3b-8efd-19f0360b5214.html",
    "02.03.01_5001": "https://cabinet.spbu.ru/Lists/1k_Rating/list_68acc7a9-eac6-4726-b4df-ade7888e212b.html",
    "02.03.01_5189": "https://cabinet.spbu.ru/Lists/1k_Rating/list_5280d407-3a61-43c8-84a8-48818ec97282.html",
    "02.03.02_5003": "https://cabinet.spbu.ru/Lists/1k_Rating/list_3bffd628-b6f7-479f-8386-ebba3f2ab5c1.html",
    "02.03.02_5190": "https://cabinet.spbu.ru/Lists/1k_Rating/list_92b01d04-a9c2-4fee-950d-bd3896abf4c4.html",
    "02.03.02_5162": "https://cabinet.spbu.ru/Lists/1k_Rating/list_c91f57a1-c31a-427a-8f57-e41bdb877a80.html",
    "03.03.00": "https://cabinet.spbu.ru/Lists/1k_Rating/list_ad180a85-bc09-41ea-9ec3-4125dd9672d4.html",
    "03.03.01_5009": "https://cabinet.spbu.ru/Lists/1k_Rating/list_dd2ad806-d66e-43ab-bb86-dfa59f6c612e.html",
    "03.03.01_5112": "https://cabinet.spbu.ru/Lists/1k_Rating/list_6349d0e3-13f3-4901-adfb-2bd9ead19134.html",
    "09.03.03_5078": "https://cabinet.spbu.ru/Lists/1k_Rating/list_c3e7f0d7-b891-4fb4-aa80-ca909a53b74b.html",
    "09.03.04_5080": "https://cabinet.spbu.ru/Lists/1k_Rating/list_73b489e3-e98f-4193-bfc7-160a9ded4532.html"
}

In [ ]:
spbu_spec = {}
for spec in spbu_dict:
    try:
        read_html = pd.read_html(spbu_dict[spec])
        spbu_spec[spec] = read_html[0]
        spbu_spec[spec]["СПбГУ"] = spec
    except Exception as e:
       print("пропустили", spbu_dict[spec], "-", e)
# len(spbu_spec)

In [ ]:
spbu_originals = []
i = 0
for spec in spbu_spec:
    spbu_originals.append(spbu_spec[spec][spbu_spec[spec]["Оригиналы"] == "Да"])
    spbu_originals[i] = spbu_originals[i].reset_index(drop=True)
# len(spbu_originals)

In [ ]:
spbu = pd.DataFrame({})
for i in spbu_originals:
    spbu = pd.concat([spbu,i], ignore_index= True, sort=False)
spbu = spbu.drop_duplicates()
spbu = spbu.reset_index(drop=True)
len(spbu)

Добавляем в нашу табличку данные об оригиналах в других вузах

In [ ]:
df_concurs_plus = pd.merge(left=df_concurs, 
                           right=hse[["СНИЛС / Уникальный идентификатор", "ВШЭ"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС / Уникальный идентификатор")
df_concurs_plus.drop(columns = ["СНИЛС / Уникальный идентификатор"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=mephi[["СНИЛС / № дела", "МИФИ"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС / № дела")
df_concurs_plus.drop(columns = ["СНИЛС / № дела"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=mipt[["СНИЛС / ИНД №", "МФТИ"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС / ИНД №")
df_concurs_plus.drop(columns = ["СНИЛС / ИНД №"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=mai[["СНИЛС/УКП", "МАИ"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС/УКП")
df_concurs_plus.drop(columns = ["СНИЛС/УКП"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=mpei[["СНИЛС или Рег.номер", "МЭИ"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС или Рег.номер")
df_concurs_plus.drop(columns = ["СНИЛС или Рег.номер"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=misis[["СНИЛС", "МИСиС"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС")
df_concurs_plus.drop(columns = ["СНИЛС"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=mirea[["СНИЛС/уникальный номер", "МИРЭА"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС/уникальный номер")
df_concurs_plus.drop(columns = ["СНИЛС/уникальный номер"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=rea[["СНИЛС", "РЭУ"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="СНИЛС")
df_concurs_plus.drop(columns = ["СНИЛС"], axis = 1, inplace=True)
df_concurs_plus = pd.merge(left=df_concurs_plus, 
                           right=miigaik[["Код", "МИИГАиК"]], 
                           how="left", 
                           left_on="СНИЛС /\rРег. номер", 
                           right_on="Код")
df_concurs_plus.drop(columns = ["Код"], axis = 1, inplace=True)
df_concurs_plus = df_concurs_plus.fillna("")
# df_concurs_plus

In [ ]:
df_concurs_plus["Оригинал в другом вузе"] = np.where(
    df_concurs_plus["Ориг."] == "Да", 
    "Нет",
    np.where(
        ((df_concurs_plus["ВШЭ"] != "") | (df_concurs_plus["МИФИ"] != "")
        | (df_concurs_plus["МФТИ"] != "") | (df_concurs_plus["МАИ"] != "")
        | (df_concurs_plus["МЭИ"] != "") | (df_concurs_plus["МИСиС"] != "")
        | (df_concurs_plus["МИРЭА"] != "") | (df_concurs_plus["РЭУ"] != "") 
        | (df_concurs_plus["МИИГАиК"] != "") 
        ), 
        "Да",
        "Не нашли"
    )
)
df_concurs_plus

In [ ]:
df_concurs_plus.to_excel(path + "конкурс" + Person + ".xlsx", index = False)

In [ ]:
len(df_concurs_plus[df_concurs_plus["Оригинал в другом вузе"] == "Да"])